<a href="https://colab.research.google.com/github/maryamdarei/CNN/blob/main/GRADCAMCNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Herbarium Data-2019


/content/drive/MyDrive/Herbarium Data-2019


In [4]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 16.1 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/Herbarium Data-2019')

In [7]:
import albumentations as A
import cv2

In [8]:
# CNN Model With PyTorch For Image Classification
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset, dataloader

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms as T
from torchvision.datasets import ImageFolder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import albumentations as A
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import cv2
import pandas as pd
import numpy as np
import utils

In [22]:
CSV_FILE = '/content/drive/MyDrive/Herbarium Data-2019/train.csv'
DATA_DIR = '/content/drive/MyDrive/Herbarium Data-2019/small-train'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
LR = 0.001
EPOCHS = 20

In [10]:
data = pd.read_csv(CSV_FILE)
data.head()

,Id,Category
0,00001.jpg,0
1,00002.jpg,0
2,00003.jpg,0
3,00004.jpg,0
4,00005.jpg,0


In [11]:
train_df, valid_df = train_test_split(data, test_size=0.2, random_state=42)

In [12]:
train_augs = A.Compose([
    A.rotate,
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(p=1),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.299, 0.224, 0.225])
])

valid_augs = A.Compose([A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.299, 0.224, 0.225])])


In [13]:
trainset = utils.ImageDataset(train_df, augs = train_augs, data_dir = DATA_DIR)
validset = utils.ImageDataset(valid_df, augs = valid_augs, data_dir = DATA_DIR)

In [14]:
image, label = validset[21]

plt.imshow(image.permute(1,2,0))
plt.title(label);

AttributeError: ignored

In [15]:
print(f"No. of examples in the {len(trainset)}")
print(f"No. of examples in the {len(validset)}")

No. of examples in the 7652
No. of examples in the 1913


**Load Dataset into Baches**

In [26]:
trainloader = dataloader.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True)
validloader = dataloader.DataLoader(validset, batch_size = BATCH_SIZE)

In [28]:
print(f"No. of examples in the {len(trainloader)}")
print(f"No. of examples in the {len(validloader)}")

No. of examples in the 479
No. of examples in the 120


In [ ]:
for images, labels in trainloader:
  break

print(f"one batch image shape {images.shape}")
print(f"one batch label shape {labels.shape}")

**Create the Model:**

In [19]:
from torch.nn.modules.pooling import MaxPool2d
class ImageModel(nn.Module):
  
    def __init__ (self):
        super (ImageModel, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = (5, 5), padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (4, 4), stride = 2),
      
            nn. Conv2d(in_channels = 16, out_channels = 16, kernel_size = (5, 5), padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (4, 4), stride = 2),

            nn. Conv2d(in_channels = 16, out_channels = 32, kernel_size = (5, 5), padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (4, 4), stride = 2),
    
            nn. Conv2d(in_channels = 32, out_channels = 64, kernel_size = (5, 5), padding = 1),
            nn.ReLU(),
            )

        self.maxpool = nn.MaxPool2d(kernel_size = (4,4), stride = 2)
  
        self.classifier = nn.Sequential(
            nn. Flatten(),
            nn. Linear (6400, 2048),
            nn.ReLU(),
            nn.Linear(2048, 3)
       )

        self.gradient = None


    def activation_hook(self, grad):
     self.gradient = grad

    def forward(self, images):
       x = self.feature_extractor(images) #activation_maps

       h = x.register_hook(self.activation_hook)

       X = self.maxpool(x)
       x = self.classifier (x)

       return x

    def get_activatin_gradient (self):  #a1, a2, a3, ... , ak
       return self.gradient

    def get_activation(self, x):  #A1, A2, A3, ... , Ak
       return self.feature_extractor(x)





In [24]:
model = ImageModel
model.to(device)

AttributeError: ignored

In [ ]:
def train_fn(dataloader, model, optimizer, ceriterion):

  model.train()
  total_loss = 0.0
  for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logits = model(images)
    loss = ceriterion(logits, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  return total_loss / len(dataloader)


In [ ]:
def eval_fn(dataloader, model, ceriterion):

  model.eval()
  total_loss = 0.0
  for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    logits = model(images)
    loss = ceriterion(logits, labels)
 

    total_loss += loss.item()

  return total_loss / len(dataloader)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR, )
ceriterion= torch.nn.CrossEntropyLoss()

In [ ]:
best_valid_loss= np.inf
for i in range(EPOCHS):
  
  train_loss = train_fn(trainloader, model, optimizer, ceriterion)
  valid_loss = eval_fn(trainloader, model, optimizer, ceriterion)

  if valid_loss < best_valid_loss:
    torch.save(model.state_dict, 'best_weights.pt')
    best_valid_loss = valid_loss
    print('SAVED_WEIGHTS_SUCCESS')

  print(f"EPOCHS :{i+1} TRAIN LOSS : {train_loss} VALID LOSS : {valid_loss}")


In [ ]:
def get_GradCAM(model. image, label, size):
  label.backward()
  gradients = model.get_activation_gradients()
  pooled_gradients = torch.mean(gradients, dim = [0, 2, 3]) #a1, a2, a3, ..., ak
  activation = model.get_activation(image).detach()  #A1, A2, A3, ... , Ak

  for i in range(activations.shape[1]):
    activations [:, i, :, :] *= pooled_gradients[i]

  heatmap = torch.mean(activations, dim = 1).squeeze().cpu()
  heatmap = nn.ReLU()(heatmap)
  heatmap /= torch.max(heatmap)
  heatmap = cv2.resize(heatmap.numpy(), (size, size))

  return heatmap




In [ ]:
image, label = validset[4]

denorm_image = image.permue(1,2,0) * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))

image = image.unsqueeze(0).to(device)

pred = model(image)
heatmap = get_gradcam(model, image, pred[0][1], size = 227 )
utils.plot_heatmap(denorm_image, pred, heatmap)
